# SHAP on BERT

*Applying SHAP's `DeepExplainer` to BERT via `pytorch_pretrained_bert`.*

The time consuming part of this project so far has been enabling the gradients to propagate the whole way through the embedding back to the individual tokens. Due to the `BertEmbeddings` layers of the `BertForSequenceClassification` model accepts the tokenized inputs as integers, which do not allow gradients in Pytorch, some re-engineering was required

----

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
import os
import torch
import shap

In [31]:
import bert.data_utils
import bert.train
import bert.sjr

import shap_additions
import display_text

---

# 1. Parameters

Training was done on a separate notebook (on collab), the parameters mirrored here to load the trained model

Setting `DO_TRAIN=False` will trigger the function to returned the trained model contained in `"./out"`

In [4]:
parameters = dict(
    DATA_DIR = "./",
    BERT_MODEL = 'bert-base-uncased',
    OUTPUT_DIR = "./out",
    CACHE_DIR = "./",
    WEIGHTS_NAME = "pytorch_model.bin",
    CONFIG_NAME = "bert_config.json",

    FP16 = False,
    TRAIN_BATCH_SIZE=32,
    EVAL_BATCH_SIZE=8,
    NUM_TRAIN_EPOCHS=1,
    WARMUP_PROPORTION=0.1,
    SEED=42,
    GRADIENT_ACCUMULATION_STEPS=1,
    LOCAL_RANK=-1,
    LEARNING_RATE=5e-5 ,
    MAX_SEQ_LENGTH=128,

    TASK_NAME = 'classification',
    DO_TRAIN=True,
    DO_EVAL=True,
    DO_LOWER_CASE=True,
)

This notebook was run offline, so set `device=cpu`

In [5]:
device='cpu'

In [6]:
data_root= os.environ['DATA'] if 'DATA' in os.environ.keys() else os.path.join(os.environ['PWD'], 'Documents/Data')

---

# 2. Load trained model, tokenizer and data

Processor is quite hacky & slow

In [8]:
imdb_proc = bert.data_utils.IMDBProcessor(data_path=data_root, device='cpu')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [10]:
parameters['DO_TRAIN'] = False
model, tokenizer = bert.train.train(imdb_proc, parameters)

07/12/2019 18:36:33 - INFO - root -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
07/12/2019 18:36:35 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\srozm\.pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/12/2019 18:36:35 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
07/12/2019 18:36:35 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2

In [11]:
raw_batch = bert.data_utils.get_test_data(imdb_proc, tokenizer, parameters, n_examples=20)

07/12/2019 18:36:57 - INFO - root -   Writing example 0 of 20
07/12/2019 18:36:57 - INFO - root -   *** Example ***
07/12/2019 18:36:57 - INFO - root -   guid: test-20952
07/12/2019 18:36:57 - INFO - root -   tokens: [CLS] * spoil ##ers - sorry ! * the first word that sprung to mind whilst watching the film is ' gr ##em ##lins ' . it ' s the only cr ##itte ##rs movie i have seen from the four movies , but i enjoyed it . it wasn ' t too complicated as i hadn ' t seen the past two , but i thought it was quite a good movie all the same . < br / > < br / > cr ##itte ##rs starts with a man , his son and daughter stopping off on the way to a vacation ( although it turns out that they end up at their own home - which i have no idea what the hell [SEP]
07/12/2019 18:36:57 - INFO - root -   input_ids: 101 1008 27594 2545 1011 3374 999 1008 1996 2034 2773 2008 22057 2000 2568 5819 3666 1996 2143 2003 1005 24665 6633 24412 1005 1012 2009 1005 1055 1996 2069 13675 27100 2869 3185 1045 2031 2464 20

---

# 3. Adjust forward behaviour
I've constructed a version of the `BertForSequenceClassification` that accepts the spase one hot encoded matricies constructed in the first part of the `BertEmbedding` class

In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [13]:
sjrmodel = bert.sjr.BertSJRModel(model, INPUT_ID_EX = raw_batch[0][:1])

In [14]:
type(model)

pytorch_pretrained_bert.modeling.BertForSequenceClassification

In [15]:
type(sjrmodel)

bert.sjr.BertSJRModel

The `SecondHalfEmbedding` is the "split open" `BertEmbeddings` class that accepts the OHE tensors

#### Additional pre-processing for sjrmodel
We have to use a helper function to get the OHE variables

In [16]:
input_ids, position_ids, token_type_ids = bert.data_utils.get_ids(raw_batch)

word_ohe = bert.data_utils.get_onehot(input_ids)
position_ohe = bert.data_utils.get_onehot(position_ids, size=512)
token_ohe = bert.data_utils.get_onehot(token_type_ids, size=2)

sjr_batch = [word_ohe, position_ohe, token_ohe]

### _Quick Check_ - are the model outputs the same?

In [17]:
bert_batch = bert.data_utils.raw_batch_to_bert(raw_batch)

In [18]:
model.eval();
sjrmodel.eval();

In [19]:
with torch.no_grad():
    bert_output = model(*bert_batch)
    sjr_output =sjrmodel(*sjr_batch)

In [20]:
bert_output[:2]

tensor([[ 1.6650, -1.9126],
        [ 1.7072, -1.9473]])

In [21]:
sjr_output[:2]

tensor([[ 1.6650, -1.9126],
        [ 1.7072, -1.9473]])

Logits match for the first couple of predictions

In [22]:
bert_output.sum()

tensor(-0.2471)

In [23]:
sjr_output.sum()

tensor(-0.2471)

And the sums of the tensors are the same - looks fine

# 4. Calculate Shap Values
These wouldn't quite work OTB hence tweaked versions are in the repo

In [24]:
explainer = shap_additions.PyTorchDeepExplainer(sjrmodel, sjr_batch)

In [28]:
shap_values = shap_additions.get_shap_values(explainer, sjr_batch)

Outer loop: 1/2
>>>> Inner loop 1/20
>>>> Inner loop 2/20
>>>> Inner loop 3/20
>>>> Inner loop 4/20
>>>> Inner loop 5/20
>>>> Inner loop 6/20
>>>> Inner loop 7/20
>>>> Inner loop 8/20
>>>> Inner loop 9/20
>>>> Inner loop 10/20
>>>> Inner loop 11/20
>>>> Inner loop 12/20
>>>> Inner loop 13/20
>>>> Inner loop 14/20
>>>> Inner loop 15/20
>>>> Inner loop 16/20
>>>> Inner loop 17/20
>>>> Inner loop 18/20
>>>> Inner loop 19/20
>>>> Inner loop 20/20
Outer loop: 2/2
>>>> Inner loop 1/20
>>>> Inner loop 2/20
>>>> Inner loop 3/20
>>>> Inner loop 4/20
>>>> Inner loop 5/20
>>>> Inner loop 6/20
>>>> Inner loop 7/20
>>>> Inner loop 8/20
>>>> Inner loop 9/20
>>>> Inner loop 10/20
>>>> Inner loop 11/20
>>>> Inner loop 12/20
>>>> Inner loop 13/20
>>>> Inner loop 14/20
>>>> Inner loop 15/20
>>>> Inner loop 16/20
>>>> Inner loop 17/20
>>>> Inner loop 18/20
>>>> Inner loop 19/20
>>>> Inner loop 20/20


# 5. Visuals
Shapley values for the negative class are taken from the positive class to combine into one score per token.
The values are divided by the absolute maximum per sequence to transform into the range (-1,1), then sized by absolute value, coloured by sentiment (if above threshold value)

In [147]:
for n in range(20):
    display_text.display_shap_text(shap_values, raw_batch, sjr_output, tokenizer, n=n, threshold=0.3)
    

## Actual Rating: <font color=red>bad</font> 

## Model prediction: <font color=green>good</font> 

##### Logits (pos, neg): [ 1.6650378 -1.9126236]

<font color='red'><font size=4.513492184202605>*</font> <font color='red'><font size=5.8372698096125815>spoil</font> <font color='red'><font size=5.959638693477004>##ers</font> <font color='red'><font size=6.854087936722452>-</font> <font color=grey><font size=4.378060452126939>sorry</font> <font color='green'><font size=4.875761972391234>!</font> <font color='red'><font size=4.869016604271747>*</font> <font color='red'><font size=4.665461877200923>the</font> <font color='green'><font size=4.909702461499755>first</font> <font color='red'><font size=5.317013817916918>word</font> <font color=grey><font size=3.978996172027417>that</font> <font color=grey><font size=3.8049544158886643>sprung</font> <font color=grey><font size=3.8447318606282406>to</font> <font color='green'><font size=5.420251334571825>mind</font> <font color=grey><font size=3.734008187318268>whilst</font> <font color=grey><font size=3.8167064506986557>watching</font> <font color=grey><font size=3.973793734744115>the</font> <font color=grey><font size=4.3453513969849675>film</font> <font color=grey><font size=4.2445943676165125>is</font> <font color=grey><font size=3.39943533924857>'</font> <font color=grey><font size=3.0586655946909262>gr</font> <font color='red'><font size=4.887417414729495>##em</font> <font color=grey><font size=3.3649733431373274>##lins</font> <font color='red'><font size=5.148981110841645>'</font> <font color='red'><font size=5.813090294509596>.</font> <font color=grey><font size=3.719797310800576>it</font> <font color=grey><font size=3.3966739765682252>'</font> <font color=grey><font size=3.5873021560858103>s</font> <font color=grey><font size=3.271256593664379>the</font> <font color=grey><font size=3.636012225127376>only</font> <font color=grey><font size=3.6498596516926445>cr</font> <font color=grey><font size=3.882563124048315>##itte</font> <font color=grey><font size=3.9264813753905106>##rs</font> <font color=grey><font size=3.5016010857836006>movie</font> <font color=grey><font size=3.953607718107131>i</font> <font color='red'><font size=5.23139720600347>have</font> <font color=grey><font size=3.2794843791910604>seen</font> <font color=grey><font size=3.979583151365183>from</font> <font color=grey><font size=3.7608845346816695>the</font> <font color=grey><font size=3.0048588529670948>four</font> <font color=grey><font size=4.4918584994386475>movies</font> <font color='red'><font size=4.596961155775207>,</font> <font color=grey><font size=3.901393447799094>but</font> <font color=grey><font size=3.053224339216257>i</font> <font color=grey><font size=3.5978067813101227>enjoyed</font> <font color='green'><font size=5.329215792649717>it</font> <font color=grey><font size=3.0432094509827228>.</font> <font color='green'><font size=6.37039494744616>it</font> <font color=grey><font size=3.398677743992288>wasn</font> <font color=grey><font size=4.103897773294935>'</font> <font color=grey><font size=4.420671583852652>t</font> <font color=grey><font size=3.2815083423833524>too</font> <font color=grey><font size=4.306470618498255>complicated</font> <font color=grey><font size=3.5055663637339602>as</font> <font color=grey><font size=4.45398206103084>i</font> <font color=grey><font size=4.1765752787869275>hadn</font> <font color=grey><font size=4.202780760960442>'</font> <font color='red'><font size=6.560040654090679>t</font> <font color=grey><font size=3.455820594380796>seen</font> <font color=grey><font size=3.1080023905029956>the</font> <font color=grey><font size=4.469621471870395>past</font> <font color=grey><font size=4.437661516279736>two</font> <font color=grey><font size=3.0552129355736857>,</font> <font color=grey><font size=3.041252980164021>but</font> <font color=grey><font size=3.981683326571737>i</font> <font color='red'><font size=5.006821297559215>thought</font> <font color='green'><font size=5.199589138842543>it</font> <font color=grey><font size=3.617863164570611>was</font> <font color=grey><font size=4.233564615480404>quite</font> <font color=grey><font size=3.8135903738668686>a</font> <font color='red'><font size=6.429426356111762>good</font> <font color=grey><font size=4.3839894938123045>movie</font> <font color='green'><font size=5.776775514426756>all</font> <font color=grey><font size=4.108612852615712>the</font> <font color='red'><font size=4.734267188118755>same</font> <font color=grey><font size=4.179788797660942>.</font> <font color='red'><font size=5.320445048465263><</font> <font color='red'><font size=5.825655072782483>br</font> <font color=grey><font size=3.681961282398149>/</font> <font color='red'><font size=4.8708824088346585>></font> <font color='red'><font size=4.779826403344926><</font> <font color='red'><font size=4.910277657664105>br</font> <font color=grey><font size=3.4755332247961466>/</font> <font color=grey><font size=3.3809761485225347>></font> <font color=grey><font size=3.0401948730077164>cr</font> <font color=grey><font size=3.31460509717476>##itte</font> <font color=grey><font size=3.7599050168364188>##rs</font> <font color='red'><font size=4.593818409817382>starts</font> <font color=grey><font size=3.861596130497456>with</font> <font color=grey><font size=4.237458418695468>a</font> <font color=grey><font size=3.864506795958196>man</font> <font color=grey><font size=3.246833279532127>,</font> <font color=grey><font size=4.3480686189373134>his</font> <font color=grey><font size=4.0332986866172025>son</font> <font color=grey><font size=4.288584647761718>and</font> <font color=grey><font size=3.1605661228622552>daughter</font> <font color=grey><font size=4.298879964672052>stopping</font> <font color='red'><font size=5.138694363511979>off</font> <font color=grey><font size=4.468476509405306>on</font> <font color=grey><font size=3.429677407887424>the</font> <font color=grey><font size=3.2793693316471746>way</font> <font color=grey><font size=4.303217354500119>to</font> <font color=grey><font size=3.259079007145523>a</font> <font color=grey><font size=3.1888028588357504>vacation</font> <font color=grey><font size=3.602606614522443>(</font> <font color=grey><font size=4.084556155856323>although</font> <font color='green'><font size=8.0>it</font> <font color='green'><font size=5.8743134473061955>turns</font> <font color='red'><font size=5.0073253514289>out</font> <font color=grey><font size=3.409996227740773>that</font> <font color=grey><font size=3.1549881523331025>they</font> <font color=grey><font size=3.6637104396020534>end</font> <font color=grey><font size=3.220154762046533>up</font> <font color=grey><font size=3.2921825123285977>at</font> <font color=grey><font size=3.3614398316887075>their</font> <font color=grey><font size=3.6668226010219844>own</font> <font color=grey><font size=4.253295077641731>home</font> <font color=grey><font size=3.3227355330038337>-</font> <font color=grey><font size=4.223223163198297>which</font> <font color='green'><font size=5.201360530266739>i</font> <font color=grey><font size=3.5082874456913906>have</font> <font color='red'><font size=5.588346983739575>no</font> <font color='red'><font size=6.351347281399527>idea</font> <font color=grey><font size=3.072682848782524>what</font> <font color=grey><font size=3.141217791969824>the</font> <font color=grey><font size=3.241200045365587>hell</font> 

---

...

## Actual Rating: <font color=red>bad</font> 

## Model prediction: <font color=red>bad</font> 

##### Logits (pos, neg): [-3.0027838  3.7282693]

<font color=grey><font size=3.283101372280587>everything</font> <font color=grey><font size=3.1953712704673327>about</font> <font color='green'><font size=4.5932739007274614>this</font> <font color=grey><font size=3.6267207481196735>movie</font> <font color=grey><font size=3.6198354227826437>is</font> <font color='red'><font size=6.027137016998648>bad</font> <font color=grey><font size=4.334723847682504>.</font> <font color='green'><font size=4.778604066774379>everything</font> <font color='green'><font size=5.117564247342704>.</font> <font color=grey><font size=3.830015726010115>ridiculous</font> <font color=grey><font size=3.244611794452446>80</font> <font color=grey><font size=3.8620890288302268>'</font> <font color=grey><font size=3.254065735992628>s</font> <font color=grey><font size=3.701810102956249>hair</font> <font color=grey><font size=3.7137719135987317>##cut</font> <font color=grey><font size=4.108417131018584>##s</font> <font color=grey><font size=3.0533226897290353>.</font> <font color='red'><font size=5.227375600541359>ridiculous</font> <font color=grey><font size=3.1651443073824193>mo</font> <font color=grey><font size=3.9624341200806166>##ust</font> <font color='green'><font size=5.109098940678721>##ache</font> <font color=grey><font size=3.456352289475653>##s</font> <font color=grey><font size=3.889044994814268>.</font> <font color='red'><font size=7.55403743974651>ridiculous</font> <font color='green'><font size=4.797119645263017>action</font> <font color=grey><font size=4.2862589808683556>and</font> <font color=grey><font size=3.7917521305783097>fight</font> <font color=grey><font size=4.204510640972763>scenes</font> <font color=grey><font size=3.0417777979846905>where</font> <font color=grey><font size=3.144117115288756>you</font> <font color=grey><font size=3.247357621732365>can</font> <font color='red'><font size=5.721468315457418>actually</font> <font color=grey><font size=3.7163801817661417>see</font> <font color=grey><font size=3.9433674568298205>that</font> <font color=grey><font size=3.0209246563527525>the</font> <font color='red'><font size=6.3225973425435145>ad</font> <font color=grey><font size=3.3988291988438526>##vers</font> <font color=grey><font size=3.4427299078546407>##aries</font> <font color=grey><font size=3.8910375474397347>do</font> <font color='red'><font size=4.614502858557357>not</font> <font color=grey><font size=3.0031922409715652>even</font> <font color=grey><font size=4.3816098995988675>hit</font> <font color='green'><font size=5.19685693572356>each</font> <font color=grey><font size=3.7741098098898602>other</font> <font color='red'><font size=4.54325662273032>.</font> <font color='red'><font size=5.542473185277984>bad</font> <font color=grey><font size=4.094056055107712>,</font> <font color='red'><font size=5.595494495557135>bad</font> <font color=grey><font size=3.4095934048483074>,</font> <font color=grey><font size=3.9284937352998113>bad</font> <font color=grey><font size=3.853574732141206>80</font> <font color=grey><font size=4.046899868800047>'</font> <font color=grey><font size=3.524970759211355>s</font> <font color='green'><font size=4.70756025942742>music</font> <font color=grey><font size=3.4088869331642027>.</font> <font color=grey><font size=3.2459415767229602>repeated</font> <font color='green'><font size=4.587480247052243>scenes</font> <font color=grey><font size=4.103853901056606>of</font> <font color=grey><font size=3.8763000860442984>people</font> <font color=grey><font size=3.8478908818662596>running</font> <font color=grey><font size=3.646940983307423>through</font> <font color=grey><font size=3.516473415509894>woods</font> <font color=grey><font size=3.2558181235034716>.</font> <font color=grey><font size=3.303171761084937>a</font> <font color=grey><font size=4.19792938251191>bad</font> <font color='red'><font size=4.654606018825403>guy</font> <font color=grey><font size=3.7688365957013987>with</font> <font color=grey><font size=3.1595074489598165>a</font> <font color=grey><font size=3.4503101731328694>silver</font> <font color='red'><font size=4.500960045432204>plastic</font> <font color=grey><font size=4.300489098595415>hand</font> <font color=grey><font size=3.5850990561340006>and</font> <font color='red'><font size=4.650574680563829>silly</font> <font color=grey><font size=3.458211020863897>hair</font> <font color=grey><font size=3.695539697868548>.</font> <font color='red'><font size=5.886957998539885>stupid</font> <font color=grey><font size=3.348768378576081>dialogue</font> <font color='red'><font size=8.0>.</font> <font color='green'><font size=4.6938299035791475>the</font> <font color=grey><font size=4.250659456635726>acting</font> <font color=grey><font size=4.381766307146334>is</font> <font color='red'><font size=4.7782935315182735>none</font> <font color=grey><font size=3.181873671914354>##xi</font> <font color=grey><font size=3.937420073092269>##stan</font> <font color=grey><font size=3.798906582935998>##t</font> <font color=grey><font size=4.273306686294347>.</font> <font color='green'><font size=5.072322171298988>everything</font> <font color=grey><font size=4.298398433371106>looks</font> <font color=grey><font size=3.931040845465186>extremely</font> <font color=grey><font size=4.176648094427998>cheap</font> <font color=grey><font size=3.390263645545844>.</font> <font color=grey><font size=3.290263459447372>this</font> <font color=grey><font size=3.574419177656409>movie</font> <font color=grey><font size=3.9635205958306816>even</font> <font color=grey><font size=3.0305895378161742>sur</font> <font color=grey><font size=3.71322387745826>##pass</font> <font color=grey><font size=3.166291311527866>##es</font> <font color=grey><font size=3.13852495831719>"</font> <font color=grey><font size=3.3633776388021017>plan</font> <font color=grey><font size=3.333047226373532>9</font> <font color=grey><font size=3.6821759719154397>from</font> <font color=grey><font size=4.108277477635523>outer</font> <font color=grey><font size=3.2984723902822792>space</font> <font color=grey><font size=4.167847041732037>"</font> <font color=grey><font size=3.0316508036187977>in</font> <font color='green'><font size=6.343169441693557>its</font> <font color='red'><font size=5.3939904636144504>utter</font> <font color='red'><font size=6.811719054464522>bad</font> <font color=grey><font size=3.284431591962039>##ness</font> <font color=grey><font size=4.261500779514009>.</font> <font color=grey><font size=4.2256585319755136><</font> <font color=grey><font size=3.7460753475916504>br</font> <font color=grey><font size=3.29828127152592>/</font> <font color=grey><font size=3.3737602766651222>></font> <font color=grey><font size=3.7371682823902113><</font> <font color=grey><font size=3.7381779858932482>br</font> <font color='red'><font size=4.573494549255292>/</font> <font color=grey><font size=3.6988497057290077>></font> <font color='green'><font size=5.380963570593226>it</font> <font color=grey><font size=3.0481116458838327>'</font> <font color=grey><font size=4.131444881683364>s</font> <font color=grey><font size=3.32147303472379>not</font> <font color=grey><font size=3.407814785936071>"</font> <font color=grey><font size=3.398126358996019>funny</font> <font color='red'><font size=5.948888600839795>bad</font> <font color=grey><font size=3.3716108923361277>"</font> 

---

# 6. Next steps
>  Model:
> - Baseline (TDIDF)
> - Add more trainable layers

> Viz:
> -  Add viz of attention
> - investigate the positions   

# 7. Fin: Save workspace

In [127]:
import shelve

In [125]:
!mkdir tmp

In [129]:
filename='./tmp/shelve.out'
my_shelf = shelve.open(filename,'n') # 'n' for new

for key in dir():
    try:
        my_shelf[key] = globals()[key]
    except:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()

ERROR shelving: Out
ERROR shelving: _12
ERROR shelving: __builtin__
ERROR shelving: __builtins__
ERROR shelving: _oh
ERROR shelving: bert
ERROR shelving: display_text
ERROR shelving: exit
ERROR shelving: explainer
ERROR shelving: get_ipython
ERROR shelving: imdb_proc
ERROR shelving: model
ERROR shelving: np
ERROR shelving: os
ERROR shelving: pd
ERROR shelving: quit
ERROR shelving: shap
ERROR shelving: shap_additions
ERROR shelving: shelve
ERROR shelving: sjrmodel
ERROR shelving: torch


To restore:

In [ ]:
my_shelf = shelve.open(filename)
for key in my_shelf:
    globals()[key]=my_shelf[key]
my_shelf.close()